In [94]:
import pandas as pd
import os

folder_path = 'energydata'

all_files = [f for f in os.listdir(folder_path) if f.endswith('_hourly.csv') and 'pjm' not in f.lower()]

data_list = []

for file in all_files:
    region_name = file.split('_')[0]
    df = pd.read_csv(os.path.join(folder_path, file))
    df['Region'] = region_name
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df.rename(columns={df.columns[1]: 'Demand'}, inplace=True)
    data_list.append(df)

combined_data = pd.concat(data_list, ignore_index=True)
combined_data.sort_values(by='Datetime', inplace=True)
combined_data.to_csv('combined_regions_hourly.csv', index=False)



In [95]:
import numpy as np

#randomly generated supply data but not in use 
np.random.seed(42)

combined_data['WindSpeed'] = np.random.normal(5, 2, len(combined_data))
combined_data['SolarRadiation'] = combined_data['Datetime'].dt.hour.apply(lambda x: 300 if 6 <= x <= 18 else 0)

combined_data['SolarProduction'] = combined_data['SolarRadiation'] * 0.5
combined_data['WindProduction'] = np.minimum(combined_data['WindSpeed'] * 2, 10)
combined_data['FossilProduction'] = np.maximum(combined_data['Demand'] - 
                                               (combined_data['SolarProduction'] + combined_data['WindProduction']), 0)


combined_data.to_csv('final_regions_with_simulation.csv', index=False)



In [96]:
import numpy as np

# base on the demxand generated the supply data and design 30% change over supply
overproduction_probability = 0.3
adjustments = np.random.choice([-1, 1], size=len(data), p=[1 - overproduction_probability, overproduction_probability])

# simulate the supply data
data['Supply'] = data['Demand'] + adjustments * np.random.uniform(0, 1000, len(data))

# add extra noise makes the simulation more natural
data['Supply'] += np.random.normal(0, 100, len(data))

#initialize price as 100
data['Price'] = 100  

# calculate the imbalance
data['Imbalance'] = data['Supply'] - data['Demand']

# define state
imbalance_bins = np.linspace(data['Imbalance'].min(), data['Imbalance'].max(), 10)
data['State'] = np.digitize(data['Imbalance'], imbalance_bins) - 1 

data[['Datetime', 'Demand', 'Supply', 'Imbalance', 'Price', 'State']].to_csv('simulated_data.csv', index=False)

print(data[['Datetime', 'Demand', 'Supply', 'Imbalance', 'Price', 'State']].head())

              Datetime     Demand      Supply   Imbalance  Price  State
0  2004-05-01 01:00:00   2.365480  193.082101  190.716621    100      5
1  2004-05-01 02:00:00   0.346681 -463.712418 -464.059099    100      2
2  2004-05-01 03:00:00  14.337037 -255.326996 -269.664032    100      3
3  2004-05-01 04:00:00   0.000000 -877.417517 -877.417517    100      1
4  2004-05-01 05:00:00  10.807845 -887.985104 -898.792948    100      1


In [111]:
data = data.iloc[:2000].reset_index(drop=True)
def adjust_price_and_demand(row, action):
    noise = np.random.normal(0, 5)  
    if action == 0:  # Increase Price
        new_price = row['Price'] * 1.1 + noise
        new_demand = max(row['Demand'] * 0.9 - noise, 0)
    elif action == 1:  # Decrease Price
        new_price = row['Price'] * 0.9 + noise
        new_demand = row['Demand'] * 1.1 + noise
    else:  # Hold Price
        new_price = row['Price']
        new_demand = row['Demand']
    return new_price, new_demand


def calculate_reward_with_price(imbalance, fossil_production, price, action):
    reward = 0

    if abs(imbalance) < 200:  
        reward += 100
    if 80 <= price <= 150:
        reward += 100
    else:
        reward -= 50
    
    if action == 0:  # Increase Price
        reward += 50 if imbalance > 50 else -10
    elif action == 1:  # Decrease Price
        reward += 50 if imbalance < -50 else -10
    elif action == 2:  # Hold Price
        reward += 50 if abs(imbalance) < 50 else -10

    return reward


#Q-Learning Agent
class QLearningAgent:
    def __init__(self, state_space, action_space, learning_rate=0.1, discount_factor=0.9, exploration_rate=0.2):
        self.state_space = state_space
        self.action_space = action_space
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate  
        self.q_table = np.zeros((state_space, action_space))  

    def get_action(self, state):
        if np.random.rand() < self.exploration_rate:  
            return np.random.randint(self.action_space)
        else:
            return np.argmax(self.q_table[state])  

    def update_q_value(self, state, action, reward, next_state, done):
        best_next_action = np.argmax(self.q_table[next_state]) 
        td_target = reward + self.discount_factor * self.q_table[next_state][best_next_action] * (1 - done)
        td_error = td_target - self.q_table[state][action]  
        self.q_table[state][action] += self.learning_rate * td_error 

state_space = len(imbalance_bins)  
action_space = 3  
agent = QLearningAgent(
    state_space=state_space,
    action_space=action_space,
    learning_rate=0.1,
    discount_factor=0.8, 
    exploration_rate=1.0  
)

num_episodes = 100
episode_rewards = []  

for episode in range(num_episodes):
    total_reward = 0

    for idx in range(len(data) - 1):
        current_row = data.iloc[idx]
        next_row = data.iloc[idx + 1]
        state = current_row['State']

        action = agent.get_action(state)

        new_price, new_demand = adjust_price_and_demand(current_row, action)
        data.at[idx, 'Price'] = new_price  # update price each step
        data.at[idx, 'Demand'] = new_demand  # update demand each step

        adjusted_supply = current_row['Supply']
        imbalance = adjusted_supply - new_demand

        reward = calculate_reward_with_price(imbalance, current_row['FossilProduction'], new_price, action)

        # get next state
        next_state = next_row['State']
        done = idx == len(data) - 2

        # update Q table
        agent.update_q_value(state, action, reward, next_state, done)
        total_reward += reward

    episode_rewards.append(total_reward)
    print(f"Episode {episode + 1}: Total Reward = {total_reward}")
    
'''
# take a look for reward trend after learning
plt.plot(range(num_episodes), episode_rewards)
plt.title("Total Rewards Over Episodes")
plt.xlabel("Episode")
plt.ylabel("Total Reward")
plt.show()

# take a look at ele-price after learning
plt.plot(data['Datetime'], data['Price'])
plt.title("Electricity Price Over Time")
plt.xlabel("Time")
plt.ylabel("Price")
plt.show()
'''


Episode 1: Total Reward = -45160
Episode 2: Total Reward = -45330
Episode 3: Total Reward = -41290
Episode 4: Total Reward = -42960
Episode 5: Total Reward = -43160
Episode 6: Total Reward = -44040
Episode 7: Total Reward = -40650
Episode 8: Total Reward = -40990
Episode 9: Total Reward = -42810
Episode 10: Total Reward = -40690
Episode 11: Total Reward = -42030
Episode 12: Total Reward = -40190
Episode 13: Total Reward = -40590
Episode 14: Total Reward = -41400
Episode 15: Total Reward = -41570
Episode 16: Total Reward = -42800
Episode 17: Total Reward = -43820
Episode 18: Total Reward = -42040
Episode 19: Total Reward = -46140
Episode 20: Total Reward = -45100
Episode 21: Total Reward = -45380
Episode 22: Total Reward = -44080
Episode 23: Total Reward = -42740
Episode 24: Total Reward = -45600
Episode 25: Total Reward = -43750
Episode 26: Total Reward = -45270
Episode 27: Total Reward = -42300
Episode 28: Total Reward = -45130
Episode 29: Total Reward = -43500
Episode 30: Total Rewar

'\n# take a look for reward trend after learning\nplt.plot(range(num_episodes), episode_rewards)\nplt.title("Total Rewards Over Episodes")\nplt.xlabel("Episode")\nplt.ylabel("Total Reward")\nplt.show()\n\n# take a look at ele-price after learning\nplt.plot(data[\'Datetime\'], data[\'Price\'])\nplt.title("Electricity Price Over Time")\nplt.xlabel("Time")\nplt.ylabel("Price")\nplt.show()\n'